In [1]:
from glob import glob
import json

In [2]:
files = glob('texts_8.jsonl*.splitted.requested')
files

['texts_8.jsonl01.splitted.requested',
 'texts_8.jsonl00.splitted.requested',
 'texts_8.jsonl07.splitted.requested',
 'texts_8.jsonl08.splitted.requested',
 'texts_8.jsonl03.splitted.requested',
 'texts_8.jsonl06.splitted.requested',
 'texts_8.jsonl02.splitted.requested',
 'texts_8.jsonl04.splitted.requested',
 'texts_8.jsonl05.splitted.requested']

In [3]:
mapping = {}
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if 'Source text\nclear\nLook up details' in l['r']['result']:
                continue
            if 'clear\nLook up details' in l['r']['result']:
                continue
            if l['r']['result'].startswith('Source text\n'):
                continue
                
            mapping[l['src']] = l['r']['result']
            
len(mapping)

888074

In [4]:
all_data = []
with open('train_8.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        data = []
        for d_ in l['data']:
            data.append({
                'value': d_,
                'value_ms': mapping.get(d_)
            })
        all_data.append(data)

In [5]:
all_data[0]

[{'value': 'March 13th saw the launch of “I Will Save You,” poems and drawings by students from all over Martha’s Vineyard.\nAll of the artwork and poems in the book were created by students from different schools around the island. The book was edited by Fan Ogilvie, educator and poet, and Richard Skidmore, Gay Head Lighthouse Keeper and poet. The book launched March 13, 2015 at Pathways in Chilmark with a poetry reading by the poets featured in the book.\nBooks will be available for purchase at island bookstores for $20 and proceeds benefit the Save the Gay Head Lighthouse project. Special thanks to Pathways Projects Institutes, Marianne Goldberg, James Weiss, Susan Stevens, Bob Moore, Amy Reece, Christine Ferrone, Todd Derry, and all the students who shared their art to help save the Light! Can you provide a summary of the "I Will Save You" book launch, including who edited the book and where it was launched?',
  'value_ms': '13 Mac menyaksikan pelancaran "Saya Akan Menyelamatkan An

In [6]:
from tqdm import tqdm
import random

In [7]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'gay',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context'
]

In [8]:
# for i in tqdm(range(len(all_data))):
#     for no, d in enumerate(all_data[i]):
#         if 'requires more context' in d['value'].lower():
#             print(d['value'])
#             break

In [9]:
from tqdm import tqdm
import random

all_texts = []
for i in tqdm(range(len(all_data))):
    texts = []
    for no, d in enumerate(all_data[i]):
        
        if no % 2 == 0:
            t = '<manusia>: '
            if random.random() > 0.5:
                c = 'value'
            else:
                c = 'value_ms'
        else:
            t = '<bot>: '
                
        if d[c] is None:
            break
        
        if any([b in d['value'].lower() for b in break_at]):
            # print(d[c], '\n-----\n')
            break
        
        n = d[c]
        n = n.strip()
        if len(n) < 3:
            break
        t = t + n
        texts.append(t)

    if len(texts) % 2 != 0:
        texts = texts[:-1]
    if not len(texts):
        continue
    while len(texts) and texts[-1].startswith('<manusia>'):
        texts = texts[:-1]
    if not len(texts):
        continue
    if not texts[0].startswith('<manusia>'):
        continue
    all_texts.append('\n'.join(texts))

100%|█████████████████████████████████| 150000/150000 [00:22<00:00, 6530.47it/s]


In [10]:
len(all_texts)

139550

In [11]:
all_texts[-2]

'<manusia>: Pameran perdagangan perubatan terbesar di dunia menawarkan peluang unik untuk menerokai inovasi terkini, mendengar penceramah yang memberi inspirasi dan menikmati rangkaian berkualiti tinggi.\nLebih 4500 peserta pameran dan 120,000 pelawat dari 120 negara dijangka berkumpul di Messe Dusseldorf pada 16-19 November 2015 untuk acara medtech tahunan. MEDICA dibahagikan kepada tema yang berbeza; elektroperubatan/teknologi perubatan, teknologi maklumat dan komunikasi, fisioterapi/peralatan ortopedik, barang pakai buang/komoditi dan barangan pengguna, peralatan makmal/diagnostik dan kumpulan sasaran pelawat. Pelawat juga boleh menggunakan sistem Siapa Siapa yang baharu, di mana mereka boleh mencari mengikut kriteria dan mencari rakan kongsi hubungan dengan mudah.\nMulai tahun ini dan seterusnya, MEDICA di Dusseldorf telah diadakan selari dengan COMPAMED, pameran perdagangan antarabangsa untuk pembekal industri teknologi perubatan. Di sini tumpuan diberikan kepada segala-galanya da

In [12]:
for t in all_texts:
    splitted = t.split('<manusia>:')
    if '<bot>:' not in splitted[-1]:
        print(t)
        break

In [13]:
with open('part8-alternate-no-alignment_texts.jsonl', 'w') as fopen:
    for t in all_texts:
        fopen.write(f'{json.dumps(t)}\n')